In [1]:
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np


from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from health.utils import load_trimaing_data, column_encoder
from joblib import dump


df = load_trimaing_data()
df = df.reset_index(drop=True)
df.set_index(['id'])


col_len = len(df.columns.to_list())
X = df.iloc[:,1:col_len -1]
Y = df.iloc[:,col_len-1:]



2024-11-15 07:57:40.024 | INFO     | health.config:<module>:11 - PROJ_ROOT path is: /Users/ceegees/workspace/iisc-learning/are-you-depressed


SyntaxError: unterminated string literal (detected at line 92) (utils.py, line 92)

In [12]:

# Transform the data
X_encoded = column_encoder.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.25, random_state=16)

# s1= set(X_k_test['Profession'].unique())
# s2 = set(X['Profession'].unique())
# print('s1-s2', s1.difference(s2))
# print('s2-s1' , s2.difference(s1))

# instantiate the model (using the default parameters)
model = LogisticRegression(random_state=16)
# # fit the model with data

model.fit(X_train, y_train)
# print(X_encoded)
# print(X_test)  
dump(model, '../models/logistic-features-not-combined-rs-16.joblib')
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)


/Users/ceegees/workspace/iisc-learning/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.92      0.96      0.94     25553
           1       0.77      0.64      0.70      5697

    accuracy                           0.90     31250
   macro avg       0.84      0.80      0.82     31250
weighted avg       0.89      0.90      0.89     31250

